동물 분류기

In [12]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf # 1.xxversion을 사용
tf.disable_v2_behavior() #2.xx version 기능 사용불가
#tf 1.xx version -> 2.xx version


#포유류? 조류? 기타?
#기준 - 털, 날개
#[털, 날개]

In [13]:
xdata= np.array(
    [[0,0],
    [1,0],
    [1,1],
    [0,0],
    [0,0],
    [0,1]]
)
ydata= np.array(
    [[1,0,0], #기타
    [0,1,0], #포유류
    [0,0,1], #조류
    [1,0,0],
    [1,0,0],
    [0,0,1]]
)

In [14]:
# hx=wx+b ->예측
# hx=wx +b ->sigmoid =>0.5 -> 분류(0,1)
# hx=w1x +b1 -> score->softmax ->확률 (기타)
#     w2x+b2 ->score ->softmax ->확률(포유류)  =>max (확률)
#     w3x+b3 -> score->softmax ->확률(조류)

In [15]:
#신경망 모델 구성
x=tf.placeholder(tf.float32, shape=[None,2])
y=tf.placeholder(tf.float32, shape=[None,3])
#분류기3개
w= tf.Variable(tf.random_uniform([2,3], -1.,1.)) #-1~1 사이일때가 모델이 잘 나오는 편..필수는 아님
#신경망은 2차원[입력층(특성), 출력층(레이블])]
b=tf.Variable(tf.zeros([3])) 
#L = score
L =tf.add(tf.matmul(x,w),b) #[None, 3]와 [3]요소간의 덧셈
L = tf.nn.relu(L)

model = tf.nn.softmax(L)
#신경망 최적화를 위한 cost함수 작성
cost= tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), axis=1))
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)


In [16]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
for step in range(1001) :
    sess.run(train, feed_dict={x:xdata, y:ydata})
#     if step%20 == 0:
#         print(step, sess.run(cost, feed_dict={x:xdata, y:ydata}))
# print(sess.run(model, feed_dict={x:xdata}))
prediction = tf.argmax(model, axis=1)
target = tf.argmax(y,1)
print("예측값", sess.run(prediction, feed_dict={x:xdata}))
print("실제값", sess.run(target, feed_dict={y:ydata}))
isCorrect = tf.equal(prediction, target)
acc= tf.reduce_mean(tf.cast(isCorrect, tf.float32))
print("정확도 : %.2f"%sess.run(acc*100, feed_dict={x:xdata, y:ydata}))

예측값 [1 1 1 1 1 0]
실제값 [0 1 2 0 0 2]
정확도 : 16.67


딥러닝 모델구성

In [24]:
xdata= np.array(
    [[0,0],
    [1,0],
    [1,1],
    [0,0],
    [0,0],
    [0,1]]
)
ydata= np.array(
    [[1,0,0], #기타
    [0,1,0], #포유류
    [0,0,1], #조류
    [1,0,0],
    [1,0,0],
    [0,0,1]])

In [25]:
x=tf.placeholder(tf.float32, shape=[None,2])
y=tf.placeholder(tf.float32, shape=[None,3])
#분류기3개
w1= tf.Variable(tf.random_uniform([2,10], -1.,1.)) 
w2= tf.Variable(tf.random_uniform([10,3], -1.,1.)) 
#신경망은 2차원[입력층(특성), 출력층(레이블])]
b1=tf.Variable(tf.zeros([10])) 
b2=tf.Variable(tf.zeros([3])) 

In [27]:
L1 =tf.add(tf.matmul(x,w1),b1)
L1 = tf.nn.relu(L1)
model=tf.add(tf.matmul(L1,w2), b2)
cost=tf.reduce_mean(
    tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=model)))
train=tf.train.AdamOptimizer(0.01).minimize(cost)